In [ ]:
from typing import Optional, List
from math import inf
from collections import defaultdict, deque
from bisect import bisect_left

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

## 例题（一）

### 	104. 二叉树的最大深度

https://leetcode.cn/problems/maximum-depth-of-binary-tree/

二叉树的深度是指，根节点到最远叶节点的最长路径上的节点个数

不要一开始就陷入细节，而是思考整棵树与其左右子树之间的关系

整棵树的最大深度 = max（左子树的最大深度，右子树的最大深度）+ 1

原问题：计算整棵树的最大深度，子问题：计算左右子树的最大深度，子问题和原问题是相似的

子问题需要把结果返回给上一级问题，因此需要使用递归

子问题的规模比原问题小，不断“递”下去，总会有个尽头，即递归的边界条件

在本题中，边界条件就是空节点，可以直接返回 0 作为答案

这个返回的过程就是递归中的“归”

---

另一个思路：递归时，除了把节点传下去，同时把路径上的节点个数也传下去

根节点这里是 1，它把 1 传下去，递归左子树，它把上面收到的 1 再加 1 得到 2，递归右子树也是一样的

递归的同时还需要维护一个全局变量，每次加完 1 之后，就更新全局变量的最大值，完成遍历后，全局变量就是答案

In [ ]:
class Solution:
    def maxDepth(self, root: Optional[TreeNode]) -> int:
        # 时间复杂度 O(n)，因为每个节点都遍历了一次
        # 空间复杂度 O(n)，因为底层使用栈存储了每次递归的结果
        if root is None:
            return 0
        return max(self.maxDepth(root.left), self.maxDepth(root.right)) + 1


class Solution:
    def maxDepth(self, root: Optional[TreeNode]) -> int:
        ans = 0
        def f(node, cnt):
            if node is None:
                return
            cnt += 1
            # 注意这里的写法
            nonlocal ans
            ans = max(ans, cnt)
            f(node.left, cnt)
            f(node.right, cnt)
        f(root, 0)
        return ans

## 课后作业

### 111. 二叉树的最小深度

https://leetcode.cn/problems/minimum-depth-of-binary-tree/

In [ ]:
class Solution:
    def minDepth(self, root: Optional[TreeNode]) -> int:
        if root is None:
            return 0
        ans = inf
        def f(node, cnt):
            nonlocal ans
            cnt += 1
            # 最优性剪枝：如果递归中发现 cnt >= ans
            # 由于继续向下递归也不会让 ans 变小，直接返回
            if cnt >= ans:
                return
            if node.left is None and node.right is None:
                ans = min(ans, cnt)
            if node.left is not None:
                f(node.left, cnt)
            if node.right is not None:
                f(node.right, cnt)
        f(root, 0)
        return ans
    

class Solution:
    def minDepth(self, root: Optional[TreeNode]) -> int:
        if root is None:
            return 0
        # root 是叶子
        if root.left is root.right:
            return 1
        # 如果子节点为空，结果设为 inf
        left_depth = self.minDepth(root.left) if root.left else inf
        right_depth = self.minDepth(root.right) if root.right else inf
        return min(left_depth, right_depth) + 1

### 112. 路径总和

https://leetcode.cn/problems/path-sum/

In [ ]:
class Solution:
    def hasPathSum(self, root: Optional[TreeNode], targetSum: int) -> bool:
        if root is None:
            return False
        s = set()
        def f(node, cnt):
            nonlocal s
            cnt += node.val
            if node.left is node.right:
                s.add(cnt)
            if node.left:
                f(node.left, cnt)
            if node.right:
                f(node.right, cnt)
        f(root, 0)
        return targetSum in s

### 129. 求根节点到叶节点数字之和

https://leetcode.cn/problems/sum-root-to-leaf-numbers/

In [ ]:
class Solution:
    def sumNumbers(self, root: Optional[TreeNode]) -> int:
        ans = list()
        def f(node, path):
            nonlocal ans
            path += str(node.val)
            if node.left is node.right:
                ans.append(int(path))
            if node.left:
                f(node.left, path)
            if node.right:
                f(node.right, path)
        f(root, '')
        return sum(ans)

### 1448. 统计二叉树中好节点的数目

https://leetcode.cn/problems/count-good-nodes-in-binary-tree/

In [ ]:
class Solution:
    def goodNodes(self, root: Optional[TreeNode]) -> int:
        ans = 0
        def f(node, n):
            if node is None:
                return
            nonlocal ans
            if node.val >= n:
                ans += 1
            n = max(node.val, n)
            f(node.left, n)
            f(node.right, n)
        f(root, -inf)
        return ans

### 987. 二叉树的垂序遍历

https://leetcode.cn/problems/vertical-order-traversal-of-a-binary-tree/

In [ ]:
class Solution:
    def verticalTraversal(self, root: Optional[TreeNode]) -> List[List[int]]:
        d = defaultdict(list)
        def f(node, row, col):
            if node is None:
                return
            d[col].append((row, node.val))
            f(node.left, row+1, col-1)
            f(node.right, row+1, col+1)
        f(root, 0, 0)
        ans = []
        for k in sorted(d.keys()):
            # 按 row 排序，row 相同则按 val 排序
            col = sorted(d[k])
            ans.append([p[1] for p in col])
        return ans

## 例题（二）

### 100. 相同的树

https://leetcode.cn/problems/same-tree/

如何定义相同？首先根节点要相同，其次左右子树要分别相同

边界条件：如果两个节点都是空，则返回 true，否则返回 false

In [ ]:
class Solution:
    def isSameTree(self, p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
        # 时间/空间复杂度都是 O(n)
        if p is None or q is None:
            return p is q
        if p.val != q.val:
            return False
        return self.isSameTree(p.left, q.left) and self.isSameTree(p.right, q.right)

### 101. 对称二叉树

https://leetcode.cn/problems/symmetric-tree/

根节点必然对称，要检查左子树和右子树是否对称

把这棵树拆成左右两棵树，看根节点是否相同，然后递归处理左的左子树和右的右子树

In [ ]:
class Solution:
    def isSymmetric(self, root: Optional[TreeNode]) -> bool:
        def f(p, q):
            if p is None or q is None:
                return p is q
            return p.val == q.val and f(p.left, q.right) and f(p.right, q.left)
        return f(root.left, root.right)

### 110. 平衡二叉树

https://leetcode.cn/problems/balanced-binary-tree/

通过递归计算左右子树的高度，要在计算高度的过程中判断是否平衡

如果在递归中发现这颗树是不平衡的，就返回 -1，父节点收到 -1 就立刻返回，不再递归

In [ ]:
class Solution:
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        def f(node):
            if node is None:
                return 0
            l = f(node.left)
            if l == -1:
                return -1
            r = f(node.right)
            if r == -1 or abs(l-r) > 1:
                return -1
            return max(l, r) + 1
        return f(root) != -1

### 199. 二叉树的右视图

https://leetcode.cn/problems/binary-tree-right-side-view/

本题一定要先递归右子树，再递归左子树

在递归的过程中，要记录当前的递归深度，如果当前深度和答案数组的长度相等，则记入答案

In [ ]:
class Solution:
    def rightSideView(self, root: Optional[TreeNode]) -> List[int]:
        ans = []
        def f(node, depth):
            nonlocal ans
            if node is None:
                return
            if depth == len(ans):
                ans.append(node.val)
            f(node.right, depth+1)
            f(node.left, depth+1)
        f(root, 0)
        return ans

## 课后作业

### 226. 翻转二叉树

https://leetcode.cn/problems/invert-binary-tree/

In [ ]:
class Solution:
    def invertTree(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        if root is not None:
            tmp = self.invertTree(root.left)
            root.left = self.invertTree(root.right)
            root.right = tmp
        return root

### 617. 合并二叉树

https://leetcode.cn/problems/merge-two-binary-trees/

In [ ]:
class Solution:
    def mergeTrees(self, root1: Optional[TreeNode], root2: Optional[TreeNode]) -> Optional[TreeNode]:
        if root1 is None:
            return root2
        if root2 is None:
            return root1
        return TreeNode(
            val=root1.val + root2.val,
            left=self.mergeTrees(root1.left, root2.left),
            right=self.mergeTrees(root1.right, root2.right),
        )

### 1026. 节点与其祖先之间的最大差值

https://leetcode.cn/problems/maximum-difference-between-node-and-ancestor/

In [ ]:
class Solution:
    def maxAncestorDiff(self, root: Optional[TreeNode]) -> int:
        ans = -1
        # 记录路径上的最大最小值即可
        def f(node, mx, mn):
            if node is None:
                return
            nonlocal ans
            ans = max(ans, mx - node.val, node.val - mn)
            mx = max(mx, node.val)
            mn = min(mn, node.val)
            f(node.left, mx, mn)
            f(node.right, mx, mn)
        f(root, root.val, root.val)
        return ans

### 1080. 根到叶路径上的不足节点

https://leetcode.cn/problems/insufficient-nodes-in-root-to-leaf-paths/

In [ ]:
class Solution:
    def sufficientSubset(self, root: Optional[TreeNode], limit: int) -> Optional[TreeNode]:
        if root is None:
            return None
        limit -= root.val
        # 对于叶节点，由于根到叶的路径仅有一条
        # 所以如果该路径的元素和小于 limit，则删除叶节点
        if root.left is root.right:
            return None if limit > 0 else root
        # 对于非叶节点，如果其子节点均被删除
        # 则说明经过当前节点的所有路径和均小于 limit，因此删除
        root.left = self.sufficientSubset(root.left, limit)
        root.right = self.sufficientSubset(root.right, limit)
        return root if root.left or root.right else None

## 例题（三）

### 98. 验证二叉搜索树

https://leetcode.cn/problems/validate-binary-search-tree/

二叉搜索树的定义是：左子树所有节点的值小于父节点的值，右子树所有节点的值大于父节点的值，左右子树也都必须是二叉搜索树

第一种做法：递归时传入节点取值的开区间范围

对于每个节点，先判断它的节点值是否在开区间内，然后再往下递归

如果对左子节点递归，就需要将开区间的右边界更新为当前节点的值，如果对右子节点递归，就更新开区间左边界

这种先访问节点值，再递归左右子树的方法叫做前序遍历

第二种做法：按照左子树-节点值-右子树的顺序遍历，看当前节点值是否大于上一个节点值，这就是中序遍历

第三种做法：做法一的前序遍历是把节点值的范围往下传，反过来也可以把节点值的范围网上传

这种做法需要先遍历左右子树，再判断节点值，这就是后序遍历

In [ ]:
# 做法一
class Solution:
    def isValidBST(self, root: Optional[TreeNode], left=-inf, right=inf) -> bool:
        if root is None:
            return True
        v = root.val
        return left < v < right and self.isValidBST(root.left, left, v) and self.isValidBST(root.right, v, right)

# 做法二
class Solution:
    pre = -inf
    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        if root is None:
            return True
        if not self.isValidBST(root.left):
            return False
        if root.val <= self.pre:
            return False
        self.pre = root.val
        return self.isValidBST(root.right)

# 做法三
class Solution:
    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        # 返回当前节点所代表的子树中所有节点值的范围
        def f(node):
            if node is None:
                # 说明当前没有值
                return inf, -inf
            l_min, l_max = f(node.left)
            r_min, r_max = f(node.right)
            x = node.val
            if x <= l_max or x >= r_min:
                # 不满足条件，直接使记录失效
                return -inf, inf
            # 这里取最小最大值是为了处理之前没有值的情况
            return min(l_min, x), max(r_max, x)
        # 检查是否因不满足条件导致记录失效
        return f(root)[1] != inf

## 课后作业

### 938. 二叉搜索树的范围和

https://leetcode.cn/problems/range-sum-of-bst/

In [ ]:
class Solution:
    def rangeSumBST(self, root: Optional[TreeNode], low: int, high: int) -> int:
        if root is None:
            return 0
        x = root.val
        if x > high:
            return self.rangeSumBST(root.left, low, high)
        if x < low:
            return self.rangeSumBST(root.right, low, high)
        return self.rangeSumBST(root.left, low, high) + x + self.rangeSumBST(root.right, low, high)

### 2476. 二叉搜索树最近节点查询

https://leetcode.cn/problems/closest-nodes-queries-in-a-binary-search-tree/

In [ ]:
class Solution:
    def closestNodes(self, root: Optional[TreeNode], queries: List[int]) -> List[List[int]]:
        # 题目没说二叉搜索树是平衡的，所以要中序遍历后二分查找
        a = []
        def f(node):
            if node is None:
                return
            f(node.left)
            a.append(node.val)
            f(node.right)
        f(root)

        ans = []
        for q in queries:
            i = bisect_left(a, q)
            if i == len(a):
                ans.append([a[i-1], -1])
            elif a[i] == q:
                ans.append([a[i], a[i]])
            elif i == 0:
                ans.append([-1, a[i]])
            else:
                ans.append([a[i-1], a[i]])
        return ans

### 230. 二叉搜索树中第 K 小的元素

https://leetcode.cn/problems/kth-smallest-element-in-a-bst/

In [ ]:
class Solution:
    def kthSmallest(self, root: Optional[TreeNode], k: int) -> int:
        def f(node):
            # 题目保证节点值非负，用 -1 表示没有找到
            if node is None:
                return -1
            # 若答案在左子树中，则直接返回
            l = f(node.left)
            if l != -1:
                return l
            #  k 减少 1，表示按照中序遍历访问到了一个节点
            nonlocal k
            k -= 1
            if k == 0:
                return node.val
            return f(node.right)
        return f(root)

### 1373. 二叉搜索子树的最大键值和

https://leetcode.cn/problems/maximum-sum-bst-in-binary-tree/

In [ ]:
class Solution:
    def maxSumBST(self, root: Optional[TreeNode]) -> int:
        # 在后序遍历验证二叉搜索树的基础上，计算当前子树全部节点之和
        ans = 0
        def f(root):
            if root is None:
                return 0, inf, -inf
            l_sum, l_min, l_max = f(root.left)
            r_sum, r_min, r_max = f(root.right)
            x = root.val
            # 如果当前子树不是二叉搜索树，那么以当前子树为左右子树的树也不会是
            if x <= l_max or x >= r_min:
                return 0, -inf, inf
            # 计算当前子树全部节点之和，更新结果
            s = l_sum + r_sum + x
            nonlocal ans
            ans = max(ans, s)
            return s, min(l_min, x), max(r_max, x)
        f(root)
        return ans

### 105. 从前序与中序遍历序列构造二叉树

https://leetcode.cn/problems/construct-binary-tree-from-preorder-and-inorder-traversal/

In [ ]:
class Solution:
    def buildTree(self, preorder: List[int], inorder: List[int]) -> Optional[TreeNode]:
        if len(preorder) == 0:
            return None
        i = inorder.index(preorder[0])
        left = self.buildTree(preorder[1:1+i], inorder[:i])
        right = self.buildTree(preorder[1+i:], inorder[i+1:])
        return TreeNode(preorder[0], left, right)

### 106. 从中序与后序遍历序列构造二叉树

https://leetcode.cn/problems/construct-binary-tree-from-inorder-and-postorder-traversal/

In [ ]:
class Solution:
    def buildTree(self, inorder: List[int], postorder: List[int]) -> Optional[TreeNode]:
        if not inorder:
            return None
        i = inorder.index(postorder[-1])
        left = self.buildTree(inorder[:i], postorder[:i])
        right = self.buildTree(inorder[i+1:], postorder[i:-1])
        return TreeNode(postorder[-1], left, right)

### 889. 根据前序和后序遍历构造二叉树

https://leetcode.cn/problems/construct-binary-tree-from-preorder-and-postorder-traversal/

In [ ]:
class Solution:
    def constructFromPrePost(self, preorder: List[int], postorder: List[int]) -> Optional[TreeNode]:
        if not preorder:
            return None
        if len(preorder) == 1:
            return TreeNode(preorder[0])
        # 存在多个答案，不妨规定在前序遍历中 preorder[1] 都是左子树的根节点值
        i = postorder.index(preorder[1])
        left = self.constructFromPrePost(preorder[1:i+2], postorder[:i+1])
        right = self.constructFromPrePost(preorder[i+2:], postorder[i+1:-1])
        return TreeNode(preorder[0], left, right)

### 1110. 删点成林

https://leetcode.cn/problems/delete-nodes-and-return-forest/

In [ ]:
class Solution:
    def delNodes(self, root: Optional[TreeNode], to_delete: List[int]) -> List[TreeNode]:
        s = set(to_delete)
        ans = []
        def f(node):
            if node is None:
                return None
            # 后序遍历
            node.left = f(node.left)
            node.right = f(node.right)
            if node.val not in s:
                return node
            # 如果当前节点被删除但子节点没有，那么子节点是一棵树的根节点
            if node.left:
                ans.append(node.left)
            if node.right:
                ans.append(node.right)
            return None
        if f(root):
            ans.append(root)
        return ans

## 例题（四）

### 236. 二叉树的最近公共祖先

https://leetcode.cn/problems/lowest-common-ancestor-of-a-binary-tree/

In [ ]:
class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        # 检查当前节点
        # 如果当前节点为 p 或 q，返回当前节点
        if root is None or root is p or root is q:
            return root
        # 否则，考虑 p 或 q 出现在子树的情况
        # 如果左右子树都有 p 或 q 出现，那么说明当前节点就是最近公共祖先
        l = self.lowestCommonAncestor(root.left, p, q)
        r = self.lowestCommonAncestor(root.right, p, q)
        if l and r:
            return root
        # 如果仅一边出现，那么返回那一边的检查结果
        if l: return l
        if r: return r
        return None

### 235. 二叉搜索树的最近公共祖先

https://leetcode.cn/problems/lowest-common-ancestor-of-a-binary-search-tree/

In [ ]:
class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        if root is None or root is p or root is q:
            return root
        if p.val < root.val and q.val < root.val:
            return self.lowestCommonAncestor(root.left, p, q)
        if p.val > root.val and q.val > root.val:
            return self.lowestCommonAncestor(root.right, p, q)
        return root

## 课后作业

### 1123. 最深叶节点的最近公共祖先

https://leetcode.cn/problems/lowest-common-ancestor-of-deepest-leaves/

In [ ]:
class Solution:
    def lcaDeepestLeaves(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        # 把每棵子树都看成子问题，对于每棵子树
        # 需要计算子树高度，找到最深叶节点的最近公共祖先
        def f(node):
            if node is None:
                return 0, None
            left_h, left_res = f(node.left)
            right_h, right_res = f(node.right)
            if left_h == right_h:
                return left_h + 1, node
            elif left_h < right_h:
                # 如果右子树更高，那么它的结果就是当前结果
                return right_h + 1, right_res
            else:
                return left_h + 1, left_res
        return f(root)[1]

## 例题（五）

### 102. 二叉树的层序遍历

https://leetcode.cn/problems/binary-tree-level-order-traversal/

In [ ]:
class Solution:
    def levelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
        if root is None:
            return []
        ans = []
        cur = [root]
        nxt = []
        while len(cur) > 0:
            level = []
            for node in cur:
                level.append(node.val)
                if node.left: nxt.append(node.left)
                if node.right: nxt.append(node.right)
            ans.append(level)
            cur = nxt
            nxt = []
        return ans

### 103. 二叉树的锯齿形层序遍历

https://leetcode.cn/problems/binary-tree-zigzag-level-order-traversal/

In [ ]:
class Solution:
    def zigzagLevelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
        if root is None:
            return []
        ans = []
        q = deque([root])
        direct = 0
        while q:
            direct += 1
            level = []
            for _ in range(len(q)):
                node = q.popleft()
                level.append(node.val)
                if direct % 2 == 1:
                    if node.left: q.append(node.left)
                    if node.right: q.append(node.right)
                else:
                    if node.right: q.append(node.right)
                    if node.left: q.append(node.left)
            ans.append(level)
            q.reverse()
        return ans

### 513. 找树左下角的值

https://leetcode.cn/problems/find-bottom-left-tree-value/

In [ ]:
class Solution:
    def findBottomLeftValue(self, root: Optional[TreeNode]) -> int:
        ans = []
        cur = deque([root])
        while cur:
            size = len(cur)
            for _ in range(size):
                node = cur.popleft()
                if node.left: cur.append(node.left)
                if node.right: cur.append(node.right)
                ans.append(node.val)
            if len(cur) == 0:
                return ans[-size]

## 课后作业

### 107. 二叉树的层序遍历 II

https://leetcode.cn/problems/binary-tree-level-order-traversal-ii/

In [ ]:
class Solution:
    def levelOrderBottom(self, root: Optional[TreeNode]) -> List[List[int]]:
        if root is None:
            return []
        ans = []
        q = deque([root])
        while q:
            level = []
            for _ in range(len(q)):
                node = q.popleft()
                level.append(node.val)
                if node.left: q.append(node.left)
                if node.right: q.append(node.right)
            ans.append(level)
        ans.reverse()
        return ans

### 116. 填充每个节点的下一个右侧节点指针

https://leetcode.cn/problems/populating-next-right-pointers-in-each-node/

In [ ]:
class Node:
    def __init__(self, val: int = 0, left: 'Node' = None, right: 'Node' = None, next: 'Node' = None):
        self.val = val
        self.left = left
        self.right = right
        self.next = next

class Solution:
    def connect(self, root: 'Optional[Node]') -> 'Optional[Node]':
        if root is None:
            return None
        q = deque([root])
        while q:
            for _ in range(len(q)):
                node = q.popleft()
                if node.left: q.append(node.left)
                if node.right: q.append(node.right)
            for i in range(len(q) - 1):
                q[i].next = q[i+1]
        return root

### 117. 填充每个节点的下一个右侧节点指针 II

https://leetcode.cn/problems/populating-next-right-pointers-in-each-node-ii/

In [ ]:
class Solution:
    def connect(self, root: 'Node') -> 'Node':
        pre = []
        def dfs(node: 'Node', depth: int) -> None:
            if node is None:
                return
            if depth == len(pre):  # node 是这一层最左边的节点
                pre.append(node)
            else:  # pre[depth] 是 node 左边的节点
                pre[depth].next = node  # node 左边的节点指向 node
                pre[depth] = node
            dfs(node.left, depth + 1)
            dfs(node.right, depth + 1)
        dfs(root, 0)  # 根节点的深度为 0
        return root

### 2415. 反转二叉树的奇数层

https://leetcode.cn/problems/reverse-odd-levels-of-binary-tree/

In [ ]:
class Solution:
    def reverseOddLevels(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        if root is None:
            return None
        q = deque([root])
        depth = 0
        while q:
            ns = []
            vs = []
            for _ in range(len(q)):
                node = q.popleft()
                ns.append(node)
                vs.append(node.val)
                if node.left: q.append(node.left)
                if node.right: q.append(node.right)
            vs.reverse()
            if depth % 2 == 1:
                for (node, v) in zip(ns, vs):
                    node.val = v
            depth += 1
        return root

### 2641. 二叉树的堂兄弟节点 II

https://leetcode.cn/problems/cousins-in-binary-tree-ii/

In [ ]:
class Solution:
    def replaceValueInTree(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        if root is None:
            return None
        root.val = 0
        q = deque([root])
        while q:
            for _ in range(len(q)):
                node = q.popleft()
                if node and not node.left is node.right:
                    q.append(node.left)
                    q.append(node.right)
            v = list(map(lambda n: n.val if n else 0, q))
            s = sum(v)
            for i, node in enumerate(q):
                if node:
                    if i % 2 == 0:
                        node.val = s - v[i] - v[i+1]
                    else:
                        node.val = s - v[i] - v[i-1]
        return root